# Example: hycom Data Processing

This notebook outlines some of the basic concepts needed for defining a C3-based achitecture to archive and work with Hycom FMRC data.

## References
https://www.hycom.org/data/gomu0pt04/expt-90pt1m000  
https://www.unidata.ucar.edu/software/tds/current/tutorial/files/FmrcPoster.pdf  
https://tds.hycom.org/thredds/catalog/GOMu0.04/expt_90.1m000/FMRC/runs/catalog.xml

## Background and Goals

The cells below download and access a single run from the hycom simulation for the Gulf of Mexico called "GOMu0.04_901m000_FMRC". FMRC means: Forcast Model Run Collection. The files retrived are in NetCDF format.  [NetCDF](https://www.unidata.ucar.edu/software/netcdf/) is a binary file format (spec/api/library) written on top of the more general [HDF5](https://www.hdfgroup.org/solutions/hdf5/) library.

### Inital Goals
1. Define a type that Mixes `File` and/ or `Client` where Hycom sim data can be collected. We have a initial prototype provisioned called `HycomFMRC`
2. Define a type to handle the data download, possibly mixing the "REST" type.
  - Do file introspection (of NetCDF/HDF5 file) to populate fields of our `HycomFMRC` type once the file is downloaded.
  - Automate retrieval using Cron etc.
3. Explore possibilities for retrieving data from files:
  - One use case: Retrieve a series of 2D slices...over time (say surface temp or something) and be able to either directly load them or stream them.
  
Generally, after solving the storage issue and figuring out source/tranform and entiy types... I am _assuming_ we will want to support the ability to retrive and/or stream data from any one of the datasets(variables) in the collection of runs _across time_.
  
### More on NetCDF
NetCDF files are HDF5 files.  These formats both have rich software ecosystem that support accessing data efficiently and are used to manage large multidimensionall datasets for many large scale HPC-based codes.  IF one were to support the use case I mentioned above using NetCDF/HDF5 only it could be accomplished as follows:
* Create a directory containing the collection of FMRC run files
* Add a "parent" file that contains a dataset that points to  each dataset in the individual run files
* Use the netcdf (or HDF5) library to open the parent file and request an array that does any sort of sliceing and dicing across all the files on desires.


## Requirements
This Notebook requires the py-hycom_1_0_0 kernel.

A prototype `HycomFMRC` type is provision wit hthe `dti-jupyter` package:

In [1]:
from datetime import date
from datetime import timedelta
import xml.etree.ElementTree as ET
import netCDF4 as nc
import requests
import pandas as pd
from pivottablejs import pivot_ui
import xmltodict
from urllib.parse import urlencode,urljoin
import pandas as pd
from IPython.display import display

## Types
The following types are currently provisioned to support Hycom Data:  
(todo: run query to list all Types in hycom- package.)  
```
HycomDataset
HycomFMRC
HycomFMRCFile
GeospatialCoverage
```
Uncomment and run help command cells below for more info.

In [2]:
#help(c3.HycomDataset)

In [3]:
#help(c3.HycomFMRC)

In [4]:
#help(c3.HycomFMRCFile)

In [5]:
# Ensure we have a Dataset entry for the desired catalog
cat_url = "https://tds.hycom.org/thredds/catalog/GOMu0.04/expt_90.1m000/FMRC/runs/catalog.xml"
gom_dataset = c3.HycomDataset.upsertHycomDatasetFromCatalog(url = cat_url)

In [6]:
# Grab the HycomDataset record that was created.
objs = c3.HycomDataset.fetch().objs
if objs:
    display(pd.DataFrame(objs.toJson()))


,type,id,name,meta,version,hycom_version,description,geospatialCoverage,catalog_url
0,HycomDataset,GOMu0.04_901m000_FMRC_1.0.1,GOMu0.04_901m000_FMRC_1.0.1,"{'type': 'Meta', 'tenantTagId': 155, 'tenant':...",1,1.0.1,HYCOM + NCODA Gulf of Mexico 1/25° Analysis (N...,"{'type': 'GeospatialCoverage', 'start': {'type...",https://tds.hycom.org/thredds/catalog/GOMu0.04...


In [7]:
# Create HycomFMRC records for every run that is currenty listed in the catalog
# This uses the...
fmrcs = gom_dataset.upsertFMRCFromDatasetCatalog()
fmrcs

c3.Arry<HycomFMRC>([c3.HycomFMRC(
  id='GOMu0.04/expt_90.1m000/FMRC/runs/GOMu0.04_901m000_FMRC_RUN_2021-08-29T12:00:00Z',
  meta=c3.Meta(
         created=datetime.datetime(2021, 8, 30, 21, 46, 39, tzinfo=datetime.timezone.utc),
         updated=datetime.datetime(2021, 8, 30, 21, 46, 39, tzinfo=datetime.timezone.utc),
         timestamp=datetime.datetime(2021, 8, 30, 21, 46, 39, tzinfo=datetime.timezone.utc)),
  version=1),
 c3.HycomFMRC(
  id='GOMu0.04/expt_90.1m000/FMRC/runs/GOMu0.04_901m000_FMRC_RUN_2021-08-28T12:00:00Z',
  meta=c3.Meta(
         created=datetime.datetime(2021, 8, 30, 21, 46, 43, tzinfo=datetime.timezone.utc),
         updated=datetime.datetime(2021, 8, 30, 21, 46, 43, tzinfo=datetime.timezone.utc),
         timestamp=datetime.datetime(2021, 8, 30, 21, 46, 43, tzinfo=datetime.timezone.utc)),
  version=1),
 c3.HycomFMRC(
  id='GOMu0.04/expt_90.1m000/FMRC/runs/GOMu0.04_901m000_FMRC_RUN_2021-08-27T12:00:00Z',
  meta=c3.Meta(
         created=datetime.datetime(2021, 8, 

In [8]:
# Grab the HycomFMRC records that were created.
objs = c3.HycomFMRC.fetch().objs
if objs:
    display(pd.DataFrame(objs.toJson()))

,type,id,meta,version,dataset,run,urlPath,timeCoverage,geospatialCoverage
0,HycomFMRC,GOMu0.04/expt_90.1m000/FMRC/runs/GOMu0.04_901m...,"{'type': 'Meta', 'tenantTagId': 155, 'tenant':...",1,"{'type': 'HycomDataset', 'id': 'GOMu0.04_901m0...",GOMu0.04_901m000_FMRC_RUN_2021-08-24T12:00:00Z,GOMu0.04/expt_90.1m000/FMRC/runs/GOMu0.04_901m...,"{'type': 'TimeRange', 'start': '2021-08-24T12:...","{'type': 'GeospatialCoverage', 'start': {'type..."
1,HycomFMRC,GOMu0.04/expt_90.1m000/FMRC/runs/GOMu0.04_901m...,"{'type': 'Meta', 'tenantTagId': 155, 'tenant':...",1,"{'type': 'HycomDataset', 'id': 'GOMu0.04_901m0...",GOMu0.04_901m000_FMRC_RUN_2021-08-25T12:00:00Z,GOMu0.04/expt_90.1m000/FMRC/runs/GOMu0.04_901m...,"{'type': 'TimeRange', 'start': '2021-08-25T12:...","{'type': 'GeospatialCoverage', 'start': {'type..."
2,HycomFMRC,GOMu0.04/expt_90.1m000/FMRC/runs/GOMu0.04_901m...,"{'type': 'Meta', 'tenantTagId': 155, 'tenant':...",1,"{'type': 'HycomDataset', 'id': 'GOMu0.04_901m0...",GOMu0.04_901m000_FMRC_RUN_2021-08-26T12:00:00Z,GOMu0.04/expt_90.1m000/FMRC/runs/GOMu0.04_901m...,"{'type': 'TimeRange', 'start': '2021-08-26T12:...","{'type': 'GeospatialCoverage', 'start': {'type..."
3,HycomFMRC,GOMu0.04/expt_90.1m000/FMRC/runs/GOMu0.04_901m...,"{'type': 'Meta', 'tenantTagId': 155, 'tenant':...",1,"{'type': 'HycomDataset', 'id': 'GOMu0.04_901m0...",GOMu0.04_901m000_FMRC_RUN_2021-08-27T12:00:00Z,GOMu0.04/expt_90.1m000/FMRC/runs/GOMu0.04_901m...,"{'type': 'TimeRange', 'start': '2021-08-27T12:...","{'type': 'GeospatialCoverage', 'start': {'type..."
4,HycomFMRC,GOMu0.04/expt_90.1m000/FMRC/runs/GOMu0.04_901m...,"{'type': 'Meta', 'tenantTagId': 155, 'tenant':...",1,"{'type': 'HycomDataset', 'id': 'GOMu0.04_901m0...",GOMu0.04_901m000_FMRC_RUN_2021-08-28T12:00:00Z,GOMu0.04/expt_90.1m000/FMRC/runs/GOMu0.04_901m...,"{'type': 'TimeRange', 'start': '2021-08-28T12:...","{'type': 'GeospatialCoverage', 'start': {'type..."
5,HycomFMRC,GOMu0.04/expt_90.1m000/FMRC/runs/GOMu0.04_901m...,"{'type': 'Meta', 'tenantTagId': 155, 'tenant':...",1,"{'type': 'HycomDataset', 'id': 'GOMu0.04_901m0...",GOMu0.04_901m000_FMRC_RUN_2021-08-29T12:00:00Z,GOMu0.04/expt_90.1m000/FMRC/runs/GOMu0.04_901m...,"{'type': 'TimeRange', 'start': '2021-08-29T12:...","{'type': 'GeospatialCoverage', 'start': {'type..."


In [9]:
# Detail: look at the timeCoverage for a single HycomFMRC
fmrcs = c3.HycomFMRC.fetch()
fmrcs.objs[0].timeCoverage

c3.TimeRange(
 start=datetime.datetime(2021, 8, 24, 12, 0, tzinfo=datetime.timezone.utc),
 end=datetime.datetime(2021, 8, 30, 0, 0, tzinfo=datetime.timezone.utc))

In [10]:
# Download a datafile for each FMRC record
# Note: currently only a fetch of a single timestep is supported, but multiple 
# files can be retrived for a single HycomFMRC record.
# This demo grabs the first available forcast time for the run.
def downloadAll(fmrcs):
    
    fmrc_files = [
        fmr.downloadFMRCRunData(
            time_start = fmr.timeCoverage.start.strftime("%Y-%m-%dT%H:%M:%SZ"),
            time_end = fmr.timeCoverage.start.strftime("%Y-%m-%dT%H:%M:%SZ")
        ) for fmr in fmrcs.objs
    ]
    return fmrc_files
        
downloadAll(fmrcs)

[c3.HycomFMRCFile(
 id='f44f0438-eeff-4136-a153-26857406f66b',
 name='GOMu0.04_901m000_FMRC_RUN_2021-08-24T12:00:00Z-2021-08-24T12:00:00Z.nc',
 meta=c3.Meta(
        created=datetime.datetime(2021, 8, 30, 21, 46, 58, tzinfo=datetime.timezone.utc),
        updated=datetime.datetime(2021, 8, 30, 21, 46, 58, tzinfo=datetime.timezone.utc),
        timestamp=datetime.datetime(2021, 8, 30, 21, 46, 58, tzinfo=datetime.timezone.utc)),
 version=1),
 c3.HycomFMRCFile(
 id='9966797f-1c6f-4582-b669-9dbee91082c8',
 name='GOMu0.04_901m000_FMRC_RUN_2021-08-25T12:00:00Z-2021-08-25T12:00:00Z.nc',
 meta=c3.Meta(
        created=datetime.datetime(2021, 8, 30, 21, 47, 11, tzinfo=datetime.timezone.utc),
        updated=datetime.datetime(2021, 8, 30, 21, 47, 11, tzinfo=datetime.timezone.utc),
        timestamp=datetime.datetime(2021, 8, 30, 21, 47, 11, tzinfo=datetime.timezone.utc)),
 version=1),
 c3.HycomFMRCFile(
 id='152e5252-22ef-4f1b-a982-7a6936fbab5b',
 name='GOMu0.04_901m000_FMRC_RUN_2021-08-26T12:00

In [11]:
# List the resulting HycomFMRCFile records
objs = c3.HycomFMRCFile.fetch().objs
if objs:
    display(pd.DataFrame(objs.toJson()))

,type,url,id,name,meta,version,hycomFMRC,timeCoverage,fileType
0,HycomFMRCFile,hycom-data/GOMu0.04_901m000_FMRC_RUN_2021-08-2...,06508501-5c89-4bb6-97d1-5858296ac7fd,GOMu0.04_901m000_FMRC_RUN_2021-08-28T12:00:00Z...,"{'type': 'Meta', 'tenantTagId': 155, 'tenant':...",1,"{'type': 'HycomFMRC', 'id': 'GOMu0.04/expt_90....","{'type': 'TimeRange', 'start': '2021-08-28T12:...",netcdf4
1,HycomFMRCFile,hycom-data/GOMu0.04_901m000_FMRC_RUN_2021-08-2...,152e5252-22ef-4f1b-a982-7a6936fbab5b,GOMu0.04_901m000_FMRC_RUN_2021-08-26T12:00:00Z...,"{'type': 'Meta', 'tenantTagId': 155, 'tenant':...",1,"{'type': 'HycomFMRC', 'id': 'GOMu0.04/expt_90....","{'type': 'TimeRange', 'start': '2021-08-26T12:...",netcdf4
2,HycomFMRCFile,hycom-data/GOMu0.04_901m000_FMRC_RUN_2021-08-2...,43b0827f-df55-45ba-b972-6adbaf77d3b0,GOMu0.04_901m000_FMRC_RUN_2021-08-29T12:00:00Z...,"{'type': 'Meta', 'tenantTagId': 155, 'tenant':...",1,"{'type': 'HycomFMRC', 'id': 'GOMu0.04/expt_90....","{'type': 'TimeRange', 'start': '2021-08-29T12:...",netcdf4
3,HycomFMRCFile,hycom-data/GOMu0.04_901m000_FMRC_RUN_2021-08-2...,9966797f-1c6f-4582-b669-9dbee91082c8,GOMu0.04_901m000_FMRC_RUN_2021-08-25T12:00:00Z...,"{'type': 'Meta', 'tenantTagId': 155, 'tenant':...",1,"{'type': 'HycomFMRC', 'id': 'GOMu0.04/expt_90....","{'type': 'TimeRange', 'start': '2021-08-25T12:...",netcdf4
4,HycomFMRCFile,hycom-data/GOMu0.04_901m000_FMRC_RUN_2021-08-2...,c8d7b8da-3dcc-40c5-baa8-b56e5c666de3,GOMu0.04_901m000_FMRC_RUN_2021-08-27T12:00:00Z...,"{'type': 'Meta', 'tenantTagId': 155, 'tenant':...",1,"{'type': 'HycomFMRC', 'id': 'GOMu0.04/expt_90....","{'type': 'TimeRange', 'start': '2021-08-27T12:...",netcdf4
5,HycomFMRCFile,hycom-data/GOMu0.04_901m000_FMRC_RUN_2021-08-2...,f44f0438-eeff-4136-a153-26857406f66b,GOMu0.04_901m000_FMRC_RUN_2021-08-24T12:00:00Z...,"{'type': 'Meta', 'tenantTagId': 155, 'tenant':...",1,"{'type': 'HycomFMRC', 'id': 'GOMu0.04/expt_90....","{'type': 'TimeRange', 'start': '2021-08-24T12:...",netcdf4


In [12]:
files = c3.FileSystem.inst().listFiles("hycom-data")
files

c3.ListFilesResult(
 files=c3.Arry<File>([c3.AzureFile(
          contentLength=39432679,
          contentLocation='fs/dti/mpodolsky/hycom-data/GOMu0.04_901m000_FMRC_RUN_2021-08-24T12:00:00Z-2021-08-24T12:00:00Z.nc',
          eTag='"0x8D96BFFB157FEC3"',
          lastModified=datetime.datetime(2021, 8, 30, 21, 46, 58, tzinfo=datetime.timezone.utc),
          contentMD5='2q9AfDVCETaaf6By+K4EqQ==',
          hasMetadata=False,
          url='azure://dev-dti/fs/dti/mpodolsky/hycom-data/GOMu0.04_901m000_FMRC_RUN_2021-08-24T12:00:00Z-2021-08-24T12:00:00Z.nc',
          blobType='BLOCK_BLOB'),
         c3.AzureFile(
          contentLength=39376074,
          contentLocation='fs/dti/mpodolsky/hycom-data/GOMu0.04_901m000_FMRC_RUN_2021-08-25T12:00:00Z-2021-08-25T12:00:00Z.nc',
          eTag='"0x8D96BFFB8CB2BF6"',
          lastModified=datetime.datetime(2021, 8, 30, 21, 47, 11, tzinfo=datetime.timezone.utc),
          contentMD5='EXMCt9pfs+oVbIMnMYJMWA==',
          hasMetadata=False,
     

In [13]:
# ToDoOpen a file to confirm...
# Question: How do I call member functions of type "File" from HycomRMRCFile?
file = c3.HycomFMRCFile.fetch().objs[0]
file.directoryUrl()

500 - NotClassified - c3.love.exceptions.C3RuntimeException_wrapIt [1560.41258]
message: "wrapped ClassCastException: c3.type.metadata.impl.PersistableImpl cannot be cast to c3.type.file.ContentMeta"
JSON: {"this": {"type": "HycomFMRCFile", "url": "hycom-data/GOMu0.04_901m000_FMRC_RUN_2021-08-28T12:00:00Z-2021-08-28T12:00:00Z.nc", "id": "06508501-5c89-4bb6-97d1-5858296ac7fd", "name": "GOMu0.04_901m000_FMRC_RUN_2021-08-28T12:00:00Z-2021-08-28T12:00:00Z.nc", "meta": {"type": "Meta", "tenantTagId": 155, "tenant": "dti-jupyter", "tag": "tc02", "created": "2021-08-30T21:47:48+00:00", "createdBy": "podolsky@berkeley.edu", "updated": "2021-08-30T21:47:48+00:00", "updatedBy": "podolsky@berkeley.edu", "timestamp": "2021-08-30T21:47:48+00:00", "fetchInclude": "[]", "fetchType": "HycomFMRCFile"}, "version": 1, "hycomFMRC": {"type": "HycomFMRC", "id": "GOMu0.04/expt_90.1m000/FMRC/runs/GOMu0.04_901m000_FMRC_RUN_2021-08-28T12:00:00Z"}, "timeCoverage": {"type": "TimeRange", "start": "2021-08-28T12:00

C3RuntimeException: 500 - NotClassified - c3.love.exceptions.C3RuntimeException_wrapIt [1560.41258]
message: "wrapped ClassCastException: c3.type.metadata.impl.PersistableImpl cannot be cast to c3.type.file.ContentMeta"
JSON: {"this": {"type": "HycomFMRCFile", "url": "hycom-data/GOMu0.04_901m000_FMRC_RUN_2021-08-28T12:00:00Z-2021-08-28T12:00:00Z.nc", "id": "06508501-5c89-4bb6-97d1-5858296ac7fd", "name": "GOMu0.04_901m000_FMRC_RUN_2021-08-28T12:00:00Z-2021-08-28T12:00:00Z.nc", "meta": {"type": "Meta", "tenantTagId": 155, "tenant": "dti-jupyter", "tag": "tc02", "created": "2021-08-30T21:47:48+00:00", "createdBy": "podolsky@berkeley.edu", "updated": "2021-08-30T21:47:48+00:00", "updatedBy": "podolsky@berkeley.edu", "timestamp": "2021-08-30T21:47:48+00:00", "fetchInclude": "[]", "fetchType": "HycomFMRCFile"}, "version": 1, "hycomFMRC": {"type": "HycomFMRC", "id": "GOMu0.04/expt_90.1m000/FMRC/runs/GOMu0.04_901m000_FMRC_RUN_2021-08-28T12:00:00Z"}, "timeCoverage": {"type": "TimeRange", "start

In [ ]:
# Cleanup
print(f"Removed {c3.HycomFMRCFile.removeAll()} HycomFMRCFile records.")
print(f"Removed {c3.HycomFMRC.removeAll()} HycomFMRC records.")
print(f"Removed {c3.HycomDataset.removeAll()} HycomDataset records")
files = c3.FileSystem.inst().listFiles("hycom-data")
if files.files:
    print(f"Deleting {len(files.files)} files")
    c3.FileSystem.inst().deleteFilesBatch(files.files)
print("Done.")